In [1]:
1+1

2

In [4]:
import os
import logging
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END


# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 로깅 설정을 구성합니다.
# LangChain의 내부 동작을 자세히 보고 싶을 때 유용합니다.
logging.getLogger("langchain").setLevel(
    logging.DEBUG
)  # LangChain 관련 로그만 DEBUG 레벨로 설정
logging.getLogger("langgraph").setLevel(
    logging.DEBUG
)  # LangGraph 관련 로그도 DEBUG 레벨로 설정


# -----------------------------------------------------------------
# 1. 도구(Tools) 정의
# -----------------------------------------------------------------
def add_two(x: str) -> str:
    """입력된 숫자에 2를 더하는 간단한 도구입니다."""
    try:
        return str(float(x) + 2)
    except Exception:
        return "도구 실행 실패: 숫자만 입력해 주세요."


def population_lookup(city: str) -> str:
    """도시 이름을 받아 인구를 조회하는 도구입니다."""
    data = {"서울": "9,515,000명 (2023년 기준)", "부산": "3,343,000명 (2023년 기준)"}
    # 일부러 '대구'에 대한 데이터를 누락하여 에러를 발생시킵니다.
    if city in data:
        return data[city]
    else:
        # 에이전트가 이 에러를 보고, 도구 사용이 실패했음을 인지하게 됩니다.
        raise ValueError(
            f"'{city}'의 인구 정보를 찾을 수 없습니다."
        )


# -----------------------------------------------------------------
# 2. 상태(State) 및 그래프 구성 요소 정의
# -----------------------------------------------------------------
class AgentState(TypedDict):
    """그래프의 상태를 나타냅니다."""

    messages: List[BaseMessage]


# 에이전트가 사용할 도구 리스트
tools = [
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="대한민국 주요 도시의 인구를 조회합니다. 도시 이름을 인수로 받습니다. (예: 서울, 부산)",
    ),
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
]

# LLM 정의 (verbose=True로 LLM의 내부 추론 과정을 출력합니다)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, verbose=True)


def create_llm_fallback():
    """순수 LLM으로만 응답을 생성하는 노드입니다."""

    def llm_only_response(state: AgentState) -> AgentState:
        print("\n--- FALLBACK TO LLM ---")
        # 대화 기록 전체를 컨텍스트로 제공하여 더 나은 답변을 유도합니다.
        history = state["messages"]

        # LLM에게 직접 질문하여 응답을 생성합니다.
        response = llm.invoke(
            history
            + [
                HumanMessage(
                    "위 대화 내용과 오류를 바탕으로, 사용자에게 친절하게 상황을 설명하고 답변해주세요."
                )
            ]
        )

        # 생성된 응답을 메시지 리스트에 추가하여 상태를 업데이트합니다.
        return {"messages": state["messages"] + [response]}

    return llm_only_response


# -----------------------------------------------------------------
# 3. 그래프(Graph) 정의 및 연결
# -----------------------------------------------------------------

# ReAct 에이전트 생성 (debug=True로 LangGraph의 실행 흐름을 자세히 출력합니다)
react_agent = create_react_agent(model=llm, tools=tools, debug=True)


def should_fallback(state: AgentState) -> str:
    """
    마지막 메시지가 도구 실행 오류인지 확인하여 다음 단계를 결정하는 라우터 함수입니다.
    """
    print("\n--- [분기 확인] 다음 경로를 결정합니다. ---")
    last_message = state["messages"][-1]

    # 마지막 메시지가 도구 호출(ToolMessage)이고, 그 내용에 에러가 포함되어 있다면 'fallback'을 반환합니다.
    if isinstance(last_message, ToolMessage) and last_message.content.startswith(
        "Error:"
    ):
        print("-> 도구 실행 오류 감지. Fallback 경로로 이동합니다.")
        return "fallback"

    # 에이전트가 최종 답변을 생성했거나, 다른 정상적인 도구 호출이면 종료합니다.
    print("-> 정상 흐름. 그래프를 종료합니다.")
    return "end"


# 워크플로우를 그래프로 정의합니다.
graph = StateGraph(AgentState)

# 그래프에 노드를 추가합니다.
graph.add_node("agent", react_agent)
graph.add_node("llm_fallback", create_llm_fallback())

# 그래프의 시작점을 설정합니다.
graph.set_entry_point("agent")

# --- 조건부 엣지(Conditional Edge) 설정 ---
# 'agent' 노드 실행 후, should_fallback 함수를 호출하여 다음 경로를 동적으로 결정합니다.
graph.add_conditional_edges(
    "agent",
    should_fallback,
    {
        # should_fallback이 "fallback"을 반환하면 "llm_fallback" 노드로 이동
        "fallback": "llm_fallback",
        # should_fallback이 "end"를 반환하면 그래프 종료
        "end": END,
    },
)
# fallback 노드 실행 후에는 그래프를 종료합니다.
graph.add_edge("llm_fallback", END)

# 정의된 그래프를 실행 가능한 애플리케이션으로 컴파일합니다.
app = graph.compile()


# -----------------------------------------------------------------
# 4. 에이전트 실행 및 테스트
# -----------------------------------------------------------------

# 에이전트가 무한 루프에 빠지는 것을 방지하기 위해 최대 반복 횟수를 설정합니다.
max_iterations = 3
recursion_limit = 2 * max_iterations + 1

# '대구'는 population_lookup 도구에 없는 도시이므로, 의도적으로 에러가 발생합니다.
# 수정된 그래프는 이 에러를 감지하고 'llm_fallback' 노드로 라우팅해야 합니다.
inputs = {"messages": [HumanMessage(content="대구의 인구는 몇 명이야?")]}

print(f"--- 에이전트 실행 시작 (최대 반복 {max_iterations}회) ---")
response = app.invoke(inputs, {"recursion_limit": recursion_limit})


# 최종 결과 출력
print("\n\n--- 최종 대화 기록 ---")
if response and "messages" in response:
    for m in response["messages"]:
        print(f"[{m.__class__.__name__}]: {getattr(m, 'content', repr(m))}")
else:
    print("최종 응답을 가져오지 못했습니다.")

--- 에이전트 실행 시작 (최대 반복 3회) ---
[values] {'messages': [HumanMessage(content='대구의 인구는 몇 명이야?', additional_kwargs={}, response_metadata={}, id='22e232d4-49eb-4b57-b425-ec6f4349dec4')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CQXzdIVT1qAO10GYqEA0iJtW', 'function': {'arguments': '{"__arg1":"대구"}', 'name': 'PopulationLookup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 112, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpbowalY6KQSkPyMn2Q9BHHmfyUxO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--487ab7df-ee6f-4d0e-99a9-949a2827d2e9-0', tool_calls=[